# Asphalt mix prediction after plastic addition

In [7]:
#%% IMPORTS
import types
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_columns', None)

## 1. Data Import 

In [2]:
#%%DATA READING AND INITIAL PREPROCESSING
 
#It returns the dataframe of interes based on the property - 'AirVoids', 'MS', 'MF', 'ITS', 'TSR'
def returnDf (propertyOfInterest):
    df = pd.read_excel('fileML.xlsx', sheet_name = propertyOfInterest, engine='openpyxl')
    df = df.set_index(propertyOfInterest + ' ID')
    df.loc[:,:'Units'] = df.loc[:,:'Units'].applymap(str)
    df.loc[:,:'Units'] = df.loc[:,:'Units'] .applymap(str.strip)
    df.replace('NS', np.nan, inplace = True)
    numericColumns = ['Aggregate absorption [%]',
                  'Apparent specific gravity',
                    0.075,
                    0.3,
                    0.6,
                    2.36,
                    4.75,
                    9.5,
                    12.5,
                    19,
                    'Plastic particle size (mm)',
                    'Mixing speed (RPM)',
                    'Mixing Temperature',
                    'Mixing Time (hours)',
                    'Plastic Addition by bitumen weight (%)',
                    ]
                    
    categoricalColumns = ['Modified asphaly Mix?',
                      'Agreggate Type',
                    'Aggregate absorption [%]',
                    'Filler used',
                    'Bitumen Type Penetration Grade',
                    'New Plastic Type',
                    'Plastic pretreatment',
                    'Plastic shape',
                    'Plastic Size',
                    'Mixing Process',
                    'Plastic melted previous to addition?',
                    'Aggregates replacement ?',
                    'Bitumen replacement?',
                    'Filler replacement',
                    'Property',
                    'Units']
    df[numericColumns] = df[numericColumns].replace('N/a', 0).astype(float)
    return df

In [3]:
dfAirVoids = returnDf('AirVoids')
dfMS = returnDf('AirVoids')
dfMF = returnDf('MF')
dfITS = returnDf('ITS')
dfTSR = returnDf('TSR')

In [8]:
dfAirVoids.describe(include = "all")

,Global ID,Modified asphaly Mix?,Agreggate Type,Aggregate absorption [%],Apparent specific gravity,0.075,0.3,0.6,2.36,4.75,9.5,12.5,19,Filler used,Bitumen Type Penetration Grade,New Plastic Type,Plastic pretreatment,Plastic shape,Plastic Size,Plastic particle size (mm),Mixing Process,Plastic melted previous to addition?,Mixing speed (RPM),Mixing Temperature,Mixing Time (hours),Aggregates replacement ?,Bitumen replacement?,Filler replacement,Plastic Addition by bitumen weight (%),Property,Units,Bitumen content in the sample,Air voids of the sample (%)
count,305,305,205,237.000000,79.000000,269.000000,269.00000,266.000000,273.000000,273.000000,266.000000,258.000000,273.000000,132,305,281,305,299,261,264.000000,305,299,158.000000,227.000000,181.000000,305,305,305,305.000000,305,305,305.000000,305.000000
unique,305,2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,6,9,2,4,4,NaN,3,3,NaN,NaN,NaN,3,3,2,NaN,1,1,NaN,NaN
top,1457,Yes,Granite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stone dust,50/70,PE,Physical,Shredded,Fine,NaN,Dry,No,NaN,NaN,NaN,No,No,No,NaN,Air Voids,%,NaN,NaN
freq,1,241,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,119,135,241,198,95,NaN,142,161,NaN,NaN,NaN,235,226,241,NaN,305,305,NaN,NaN
mean,NaN,NaN,NaN,1.113819,2.706203,5.367472,13.03803,20.222218,36.018864,48.102527,66.324812,78.130116,91.661172,NaN,NaN,NaN,NaN,NaN,NaN,3.817008,NaN,NaN,1332.784810,115.845815,0.234144,NaN,NaN,NaN,15.360045,NaN,NaN,5.121121,4.258557
std,NaN,NaN,NaN,0.901977,0.066591,1.437309,3.44642,6.382403,7.256635,9.094616,10.704868,11.154531,7.346771,NaN,NaN,NaN,NaN,NaN,NaN,6.401008,NaN,NaN,1729.498611,74.670844,0.425352,NaN,NaN,NaN,29.316095,NaN,NaN,0.817212,1.577934
min,NaN,NaN,NaN,0.130000,2.590000,2.190000,5.30000,10.070000,20.000000,24.000000,47.000000,66.630000,83.110000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,2.311000,1.840000
25%,NaN,NaN,NaN,0.470000,2.650000,4.620000,10.15000,14.770000,29.850000,41.540000,59.850000,69.380000,84.460000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,1.000000,NaN,NaN,4.800000,3.420000
50%,NaN,NaN,NaN,0.990000,2.710000,5.280000,12.97000,19.780000,35.370000,46.150000,60.510000,78.000000,94.520000,NaN,NaN,NaN,NaN,NaN,NaN,4.750000,NaN,NaN,2000.000000,150.000000,0.030000,NaN,NaN,NaN,4.861111,NaN,NaN,5.170000,4.000000
75%,NaN,NaN,NaN,1.200000,2.740000,5.980000,14.47000,21.250000,38.800000,50.910000,70.610000,87.740000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,2000.000000,160.000000,0.250000,NaN,NaN,NaN,11.111111,NaN,NaN,5.500000,4.770000
